In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

2024-09-28 22:36:13.927672: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-28 22:36:13.928800: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-28 22:36:13.950769: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-28 22:36:13.951232: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-28 22:36:14.338028: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
# Reading the input data and targets data Simulated dataset 
# Example data: LBA, Size, r/w, Sequential
data = np.loadtxt('input_data.txt')
targets = np.loadtxt('target_data.txt')

In [3]:
# Prepare the data for GRU
X = np.expand_dims(data, axis=0)  # Adding batch dimension
y = np.expand_dims(targets, axis=0)  # Adding batch dimension

In [4]:
# Define the GRU model in TensorFlow
model = models.Sequential()
model.add(layers.GRU(10, input_shape=(None, X.shape[2]), return_sequences=True))
model.add(layers.Dense(1))

2024-09-28 22:36:22.327214: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-09-28 22:36:22.327822: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-09-28 22:36:22.328436: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [5]:
# Compile the model
model.compile(optimizer='adam', loss='mse')


In [ ]:
# Train the model (for simplicity, using a few epochs)
model.fit(X, y, epochs=50)

Epoch 1/50


2024-09-28 22:36:26.376020: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-09-28 22:36:26.377054: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-09-28 22:36:26.377648: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
# Model summary
model.summary()


In [ ]:
# Get the GRU layer weights
gru_layer = model.layers[0]
weights = gru_layer.get_weights()

In [ ]:
# Save weights to use them in the C code
np.savez('gru_weights.npz', Wz=weights[0], Uz=weights[1], bz=weights[2])
print("GRU weights saved to 'gru_weights.npz'")

In [ ]:
# Predict on the data (inference)
predictions = model.predict(X)
print("Predictions:", predictions)